# Importação, Organização e Transformação de Dados com Pandas

**Objetivo:** Apresentar de forma prática as etapas iniciais do fluxo de trabalho em Ciência de Dados usando Python e Pandas.

## 0. Configuração Inicial e Imports

Vamos importar as bibliotecas necessárias para esta aula.

In [107]:
import bs4
import pandas as pd
import numpy as np

import requests              # Para requisições HTTP (Web Scraping)
import sqlite3               # Para interagir com bancos de dados SQLite
import os                    # Para interagir com o sistema operacional (ex: verificar arquivos)
import time                  # Para pausas no scraping (boas práticas)

from bs4 import BeautifulSoup # Para parsear HTML (Web Scraping)

# Configurações do Pandas para melhor visualização (opcional)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 100)

print(f"Pandas Version: {pd.__version__}")
print(f"NumPy Version: {np.__version__}")
print(f"Requests Version: {requests.__version__}")
print(f"BeautifulSoup Version: {bs4.__version__}")
print(f"SQLite3 Version: {sqlite3.sqlite_version}")

Pandas Version: 2.2.3
NumPy Version: 2.2.5
Requests Version: 2.32.3
BeautifulSoup Version: 4.13.4
SQLite3 Version: 3.45.3


## 1. Introdução: A Base da Análise de Dados

As etapas iniciais de **Coleta, Importação, Organização e Transformação Estrutural** são cruciais. Elas definem a matéria-prima e garantem que os dados estejam em um formato adequado para análise. Utilizaremos o princípio do **Tidy Data** como guia.

<img src="resources/01-fluxo.svg" width="1100">


## 2. Coleta de Dados (Data Collection)

Esta etapa envolve identificar e obter dados de diversas fontes:
*   Bancos de Dados (Internos/Externos)
*   APIs (Web Services)
*   Arquivos (CSV, JSON, Excel, TXT, etc.)
*   Websites (via Web Scraping)
*   Sensores / IoT

Neste notebook, focaremos no processamento dos dados *após* a coleta (principalmente a partir de arquivos e web scraping simulado).

## 3. Web Scraping

Técnica para extrair dados de websites quando não há API ou arquivo disponível.

**Processo:** Requisição HTTP -> Recebimento HTML -> Parseamento -> Extração -> Armazenamento.

**Ferramentas:** `requests` (requisições), `BeautifulSoup` (parseamento).

⚠️ **IMPORTANTE: Considerações Éticas e Legais** ⚠️
*   **Verifique `robots.txt`:** Respeite as regras do site.
*   **Leia os Termos de Serviço (ToS):** Veja se o scraping é permitido.
*   **Não sobrecarregue:** Use intervalos (`time.sleep()`) entre requisições.
*   **Identifique-se:** Use um `User-Agent` descritivo.
*   **LGPD:** Cuidado com dados pessoais.

In [108]:
url = 'https://diegopatr.github.io/data-science-course/data-science-course/_attachments/tabela_dados.html'

df_scraped = pd.DataFrame() # Inicializar DataFrame vazio

print(f"Acessando a URL: {url}")

try:
    # 1. Requisição HTTP
    headers = {'User-Agent': 'MeuBotDeEstudo/1.0 (contato@exemplo.com)'} # Boa prática: identificar-se
    response = requests.get(url, timeout=10, headers=headers)
    response.raise_for_status() # Verifica erros (4xx, 5xx)
    print(f"Status Code: {response.status_code} - Conexão OK!")

    # 2. Parseamento HTML
    soup = BeautifulSoup(response.text, 'lxml')

    # 3. Extração (procurando a tabela com id='dados_principais')
    tabela = soup.find('table', id='dados_principais')

    if tabela:
        headers_scraped = [th.text.strip() for th in tabela.find_all('th')]
        rows = []
        for tr in tabela.find_all('tr')[1:]: # Pula header row
            cells = [td.text.strip() for td in tr.find_all('td')]
            if len(cells) == len(headers_scraped):
                 rows.append(cells)

        # 4. Armazenamento em DataFrame
        df_scraped = pd.DataFrame(rows, columns=headers_scraped)
        print("\nDados extraídos via Web Scraping:")
        display(df_scraped)
    else:
        print(f"Tabela com id 'dados_principais' não encontrada em {url}")

except requests.exceptions.ConnectionError as e:
    print(f"\nERRO DE CONEXÃO: Verifique se URL {url} está disponível. Detalhe: {e}")
except requests.exceptions.RequestException as e:
    print(f"\nERRO NA REQUISIÇÃO HTTP para {url}: {e}")
except Exception as e:
    print(f"\nERRO INESPERADO durante o scraping: {e}")

Acessando a URL: https://diegopatr.github.io/data-science-course/data-science-course/_attachments/tabela_dados.html
Status Code: 200 - Conexão OK!

Dados extraídos via Web Scraping:


,ID do Produto,Nome,Categoria,Preço (R$),Em Estoque
0,PROD-001,Laptop UltraBook V2,Eletrônicos,5200.50,Sim
1,PROD-002,Teclado Gamer RGB,Acessórios de PC,450.00,Sim
2,PROD-003,"Monitor Curvo 27""",Monitores,1899.99,Não
3,PROD-004,Mouse Sem Fio Ergonômico,Acessórios de PC,199.90,Sim
4,PROD-005,Cadeira Gamer Profissional,Móveis,1350.00,Sim


## 4. Importação de Dados

Carregar dados para DataFrames Pandas. O controle dos parâmetros de leitura é essencial para lidar com dados reais.

**Parâmetros Comuns:**
*   `filepath_or_buffer`: Caminho/URL do arquivo.
*   `sep`/`delimiter`: Separador de campos (CSV, TSV).
*   `header`: Linha do cabeçalho.
*   `names`: Nomes das colunas (se não houver header).
*   `dtype`: *Especificar tipos de dados* (muito recomendado!).
*   `na_values`: Valores a serem tratados como nulos (NaN).
*   `parse_dates`: Colunas a serem convertidas para data/hora.
*   `encoding`: Codificação do arquivo (`utf-8`, `latin-1`, etc.).
*   `skiprows`, `nrows`: Pular/limitar linhas.
*   `decimal`, `thousands`: Separadores numéricos não padrão.
*   `on_bad_lines`: Como tratar linhas com erro.

### 4.1. Importando CSV com Parâmetros

In [109]:
df_csv = pd.DataFrame() # Limpar/inicializar
try:
    # Definindo tipos e valores nulos explicitamente
    col_types = {
        'ID_Cliente': str,
        'Nome': str,
        'Idade': 'Int64', # Inteiro que suporta NaN
        'Cidade': str,
        # 'Data_Cadastro': str, # Deixar Pandas inferir ou parsear depois
        'Valor_Gasto': float
    }
    missing_values = ["", "NA", "N/A", "--"] # O que considerar NaN

    df_csv = pd.read_csv(
        'data/arquivo_dados.csv',
        sep=',',
        encoding='utf-8',
        header=0,              # Primeira linha é o cabeçalho
        dtype=col_types,       # Especificar tipos
        na_values=missing_values, # Definir nulos
        parse_dates=['Data_Cadastro'], # Tentar converter esta coluna para datetime
        # dayfirst=False       # Opcional: Ajuda a interpretar datas ambíguas (DD/MM vs MM/DD)
    )
    print("DataFrame carregado do CSV:")
    display(df_csv)
    print("\nInformações do DataFrame:")
    df_csv.info()

except FileNotFoundError:
    print("Erro: Arquivo 'arquivo_dados.csv' não encontrado.")
except pd.errors.ParserError as e:
     print(f"Erro de parseamento no CSV: {e}")
except Exception as e:
    print(f"Erro inesperado ao ler o CSV: {e}")

DataFrame carregado do CSV:


,ID_Cliente,Nome,Idade,Cidade,Data_Cadastro,Valor_Gasto
0,C001,Ana Silva,28,Sao Paulo,2023-01-15,150.75
1,C002,Bruno Costa,35,Rio de Janeiro,2023-02-20,89.90
2,C003,Carla Dias,41,Belo Horizonte,2023-03-10,210.00
3,C004,Daniel Souza,22,Curitiba,2023-04-05,55.50
4,C005,Elisa Rocha,30,Porto Alegre,2023-05-12,320.15



Informações do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID_Cliente     5 non-null      object        
 1   Nome           5 non-null      object        
 2   Idade          5 non-null      Int64         
 3   Cidade         5 non-null      object        
 4   Data_Cadastro  5 non-null      datetime64[ns]
 5   Valor_Gasto    5 non-null      float64       
dtypes: Int64(1), datetime64[ns](1), float64(1), object(3)
memory usage: 377.0+ bytes


### 4.2. Importando JSON

In [110]:
df_json = pd.DataFrame() # Limpar/inicializar
try:
    # 'orient=records' espera uma lista de dicionários JSON
    df_json = pd.read_json('data/arquivo_dados.json', orient='records',
                           dtype={'ID_Cliente':str, 'Quantidade':'Int64', 'Preco_Unitario':float}) # Especificar tipos!
    print("DataFrame carregado do JSON:")
    display(df_json)
    df_json.info()
except FileNotFoundError:
    print("Erro: Arquivo 'arquivo_dados.json' não encontrado.")
except ValueError as e:
    print(f"Erro ao parsear o JSON (verifique formato/orient): {e}")
except Exception as e:
    print(f"Erro inesperado ao ler o JSON: {e}")

DataFrame carregado do JSON:


,ID_Cliente,Nome,Idade,Cidade,Data_Cadastro,Valor_Gasto
0,C001,Ana Silva,28,Sao Paulo,2023-01-15,150.75
1,C002,Bruno Costa,35,Rio de Janeiro,2023-02-20,89.90
2,C003,Carla Dias,41,Belo Horizonte,2023-03-10,210.00
3,C004,Daniel Souza,22,Curitiba,2023-04-05,55.50
4,C005,Elisa Rocha,30,Porto Alegre,2023-05-12,320.15


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID_Cliente     5 non-null      object 
 1   Nome           5 non-null      object 
 2   Idade          5 non-null      int64  
 3   Cidade         5 non-null      object 
 4   Data_Cadastro  5 non-null      object 
 5   Valor_Gasto    5 non-null      float64
dtypes: float64(1), int64(1), object(4)
memory usage: 372.0+ bytes


### 4.3. Importando Excel

A leitura de arquivos Excel (`.xlsx`, `.xls`) requer a instalação da biblioteca `openpyxl` (para `.xlsx`) ou `xlrd` (para `.xls` mais antigos).

In [111]:
excel_file_path = 'data/planilha_dados.xlsx'
if os.path.exists(excel_file_path):
     df_excel = pd.DataFrame()
     try:
          df_excel = pd.read_excel(excel_file_path,
                                   sheet_name='DadosExemplo', # Nome ou índice da aba
                                   dtype={'col1': int, 'col2': float}, # Especificar tipos
                                   na_values=['Nulo'])
          print("\nDataFrame carregado do Excel:")
          display(df_excel.head())
          df_excel.info()
     except FileNotFoundError:
          print(f"Erro: Arquivo '{excel_file_path}' não encontrado.")
     except ImportError:
          print("Erro: Instale 'openpyxl' (`pip install openpyxl` ou `!pip install openpyxl`) para ler arquivos .xlsx.")
     except Exception as e:
          print(f"Erro ao ler o arquivo Excel: {e}")
else:
    print("\nArquivo Excel não existe e não pôde ser criado, pulando leitura.")


DataFrame carregado do Excel:


,ID_Cliente,Nome,Idade,Cidade,Data_Cadastro,Valor_Gasto
0,C001,Ana Silva,28,Sao Paulo,2023-01-15,150.75
1,C002,Bruno Costa,35,Rio de Janeiro,2023-02-20,89.90
2,C003,Carla Dias,41,Belo Horizonte,2023-03-10,210.00
3,C004,Daniel Souza,22,Curitiba,2023-04-05,55.50
4,C005,Elisa Rocha,30,Porto Alegre,2023-05-12,320.15


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID_Cliente     5 non-null      object 
 1   Nome           5 non-null      object 
 2   Idade          5 non-null      int64  
 3   Cidade         5 non-null      object 
 4   Data_Cadastro  5 non-null      object 
 5   Valor_Gasto    5 non-null      float64
dtypes: float64(1), int64(1), object(4)
memory usage: 372.0+ bytes


### 4.4. Importando de Banco de Dados SQLite

In [112]:
db_filename = 'output/meu_banco.db'
table_name = 'minha_tabela'

conn_read = None
df_sql = pd.DataFrame() # Limpar/inicializar
try:
    conn_read = sqlite3.connect(db_filename)
    # Query para selecionar colunas específicas com uma condição
    query = f"SELECT id, coluna1, coluna2, coluna_numerica, condicao FROM {table_name} WHERE condicao = 'valor';"
    print(f"\nExecutando query: {query}")

    # Ler dados diretamente para um DataFrame
    df_sql = pd.read_sql_query(query, conn_read,
                               index_col='id', # Usar a coluna id do SQL como índice do DataFrame
                               parse_dates=None, # Nenhuma coluna de data para parsear neste exemplo
                               dtype={'coluna_numerica': float} # Especificar tipo para garantir
                              )

    print("\nDataFrame carregado do Banco de Dados SQL:")
    display(df_sql)
    df_sql.info()

except sqlite3.Error as e:
    print(f"Erro ao executar a consulta SQL ou conectar: {e}")
except pd.io.sql.DatabaseError as e:
     print(f"Erro do Pandas ao ler SQL: {e}")
except Exception as e:
    print(f"Erro inesperado na leitura do banco de dados: {e}")
finally:
    if conn_read:
        conn_read.close()
        print("\nConexão de leitura com o banco de dados fechada.")


Executando query: SELECT id, coluna1, coluna2, coluna_numerica, condicao FROM minha_tabela WHERE condicao = 'valor';
Erro do Pandas ao ler SQL: Execution failed on sql 'SELECT id, coluna1, coluna2, coluna_numerica, condicao FROM minha_tabela WHERE condicao = 'valor';': no such table: minha_tabela

Conexão de leitura com o banco de dados fechada.


### 4.5. Importando Arquivos de Largura Fixa (FWF)

In [113]:
df_fwf = pd.DataFrame() # Limpar/inicializar
try:
    # Especificar larguras ou posições
    # widths = [3, 10, 8, 1] # IDs de 3 chars, Nome 10, Valor 8, Status 1
    col_specs = [(0, 3), (3, 13), (13, 21), (21, 22)] # [start, end)
    col_names = ['ID', 'Nome', 'Valor', 'Status']

    df_fwf = pd.read_fwf(
        'data/arquivo_largura_fixa.txt',
        colspecs=col_specs,
        names=col_names,
        header=None, # Nomes definidos em 'names'
        skiprows=1,  # Pular linha de cabeçalho original
        encoding='utf-8',
        dtype={'ID': str, 'Valor': float, 'Status': str} # Especificar tipos
    )
    print("DataFrame carregado de arquivo de largura fixa (FWF):")
    display(df_fwf)
    df_fwf.info()

except FileNotFoundError:
    print("Erro: Arquivo 'arquivo_largura_fixa.txt' não encontrado.")
except Exception as e:
    print(f"Erro ao ler o arquivo FWF: {e}")

DataFrame carregado de arquivo de largura fixa (FWF):


,ID,Nome,Valor,Status
0,001,João Silva,1234.50,NaN
1,002,Ana Souza,2500.75,NaN
2,003,Pedro Luz,750.25,NaN
3,004,Maria Jose,3421.00,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      4 non-null      object 
 1   Nome    4 non-null      object 
 2   Valor   4 non-null      float64
 3   Status  0 non-null      object 
dtypes: float64(1), object(3)
memory usage: 260.0+ bytes


### 4.6. Exportação de Dados

O Pandas permite salvar DataFrames em diversos formatos.

In [114]:
# Exemplo usando o df_csv que carregamos (se ele foi carregado com sucesso)
if not df_csv.empty:
    try:
        # Salvar em CSV com separador ; e decimal ,
        df_csv.to_csv('output/dados_exportados.csv', index=False, sep=';', decimal=',', encoding='utf-8')
        print("Arquivo 'dados_exportados.csv' salvo com sucesso (separador ';', decimal ',').")
        
        # Salvar em Excel (requer openpyxl)
        # df_csv.to_excel('dados_exportados.xlsx', index=False, sheet_name='Clientes')
        # print("Arquivo 'dados_exportados.xlsx' salvo com sucesso.")
    except Exception as e:
        print(f"Erro ao exportar dados: {e}")
else:
    print("DataFrame df_csv está vazio, exportação pulada.")

Arquivo 'dados_exportados.csv' salvo com sucesso (separador ';', decimal ',').


## 5. Organização e Transformação Estrutural (Data Tidying)

Após importar, organizamos e transformamos a estrutura dos dados para o formato **Tidy Data**, facilitando a análise.

**Princípios Tidy:**
1.  Cada variável em sua coluna.
2.  Cada observação em sua linha.
3.  Cada tipo de unidade observacional em sua tabela.

### 5.1. Inspeção Inicial

Revisitar o DataFrame `df_csv` (se carregado) para verificar estrutura e conteúdo.

In [115]:
if not df_csv.empty:
    print("Revisando df_csv:")
    print("Dimensões:", df_csv.shape)
    print("\nTipos de Dados:\n", df_csv.dtypes)
    print("\nCabeçalho:")
    display(df_csv.head())
    print("\nValores Nulos por Coluna:\n", df_csv.isnull().sum())
    print("\nLinhas Duplicadas:", df_csv.duplicated().sum())
    print("\nResumo Estatístico (Numérico):")
    display(df_csv.describe(include=[np.number]))
    print("\nResumo Estatístico (Datas):")
    display(df_csv.describe(include=['datetime64[ns]']))
else:
    print("df_csv não foi carregado ou está vazio.")

Revisando df_csv:
Dimensões: (5, 6)

Tipos de Dados:
 ID_Cliente               object
Nome                     object
Idade                     Int64
Cidade                   object
Data_Cadastro    datetime64[ns]
Valor_Gasto             float64
dtype: object

Cabeçalho:


,ID_Cliente,Nome,Idade,Cidade,Data_Cadastro,Valor_Gasto
0,C001,Ana Silva,28,Sao Paulo,2023-01-15,150.75
1,C002,Bruno Costa,35,Rio de Janeiro,2023-02-20,89.90
2,C003,Carla Dias,41,Belo Horizonte,2023-03-10,210.00
3,C004,Daniel Souza,22,Curitiba,2023-04-05,55.50
4,C005,Elisa Rocha,30,Porto Alegre,2023-05-12,320.15



Valores Nulos por Coluna:
 ID_Cliente       0
Nome             0
Idade            0
Cidade           0
Data_Cadastro    0
Valor_Gasto      0
dtype: int64

Linhas Duplicadas: 0

Resumo Estatístico (Numérico):


,Idade,Valor_Gasto
count,5.0,5.000000
mean,31.2,165.260000
std,7.190271,104.796836
min,22.0,55.500000
25%,28.0,89.900000
50%,30.0,150.750000
75%,35.0,210.000000
max,41.0,320.150000



Resumo Estatístico (Datas):


,Data_Cadastro
count,5
mean,2023-03-13 09:36:00
min,2023-01-15 00:00:00
25%,2023-02-20 00:00:00
50%,2023-03-10 00:00:00
75%,2023-04-05 00:00:00
max,2023-05-12 00:00:00


### 5.2. Renomear Colunas

Padronizar nomes de colunas.

In [116]:
if not df_csv.empty:
    df_renomeado = df_csv.copy() # Trabalhar com uma cópia

    # Método 1: Atribuição direta (bom para renomear todas, cuidado com a ordem)
    # df_renomeado.columns = ['id_cliente', 'nome_cliente', 'idade', 'cidade_residencia', 'data_registro', 'gasto_total']

    # Método 2: rename() (bom para renomear algumas)
    df_renomeado = df_renomeado.rename(columns={
        'ID_Cliente': 'id_cli', 
        'Nome': 'nome', 
        'Idade': 'idade', # Manter 'idade' como exemplo
        'Cidade': 'cidade',
        'Valor_Gasto': 'valor',
        'Data_Cadastro': 'dt_cadastro'
    })

    print("Colunas após renomear:")
    print(df_renomeado.columns)
    display(df_renomeado.head(2))
else:
    df_renomeado = pd.DataFrame() # Inicializa vazio se df_csv não existe
    print("df_csv vazio, pulando renomeação.")

Colunas após renomear:
Index(['id_cli', 'nome', 'idade', 'cidade', 'dt_cadastro', 'valor'], dtype='object')


,id_cli,nome,idade,cidade,dt_cadastro,valor
0,C001,Ana Silva,28,Sao Paulo,2023-01-15,150.75
1,C002,Bruno Costa,35,Rio de Janeiro,2023-02-20,89.90


### 5.3. Ajustar Tipos de Dados (`dtypes`)

Converter colunas para tipos apropriados, se não foram ajustados na importação.

In [117]:
if not df_renomeado.empty:
    df_tipos = df_renomeado.copy()
    print("Tipos ANTES das conversões:")
    df_tipos.info()

    # Exemplo: Se 'valor' fosse string (object), converteríamos:
    # df_tipos['valor'] = df_tipos['valor'].astype(str) # Forçar para string para o exemplo
    # df_tipos['valor'] = pd.to_numeric(df_tipos['valor'], errors='coerce')

    # Exemplo: Converter 'idade' (Int64) para float (pode perder informação se houver NaN)
    # Nota: A conversão Int64 -> float transforma <NA> em NaN
    try:
         df_tipos['idade'] = df_tipos['idade'].astype(float)
    except Exception as e:
         print(f"Erro ao converter idade para float: {e}")

    # Exemplo: Se 'dt_cadastro' fosse string, converteríamos:
    # df_tipos['dt_cadastro'] = df_tipos['dt_cadastro'].astype(str)
    # df_tipos['dt_cadastro'] = pd.to_datetime(df_tipos['dt_cadastro'], errors='coerce', format='%Y-%m-%d')

    print("\nTipos DEPOIS das conversões:")
    df_tipos.info()
    display(df_tipos.head(3))
else:
    df_tipos = pd.DataFrame()
    print("df_renomeado vazio, pulando ajuste de tipos.")

Tipos ANTES das conversões:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_cli       5 non-null      object        
 1   nome         5 non-null      object        
 2   idade        5 non-null      Int64         
 3   cidade       5 non-null      object        
 4   dt_cadastro  5 non-null      datetime64[ns]
 5   valor        5 non-null      float64       
dtypes: Int64(1), datetime64[ns](1), float64(1), object(3)
memory usage: 377.0+ bytes

Tipos DEPOIS das conversões:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_cli       5 non-null      object        
 1   nome         5 non-null      object        
 2   idade        5 non-null      float64       
 3   cidade       5

,id_cli,nome,idade,cidade,dt_cadastro,valor
0,C001,Ana Silva,28.0,Sao Paulo,2023-01-15,150.75
1,C002,Bruno Costa,35.0,Rio de Janeiro,2023-02-20,89.90
2,C003,Carla Dias,41.0,Belo Horizonte,2023-03-10,210.00


### 5.4. Reshaping: Wide vs Long Format

*   **Wide:** Observação espalhada em múltiplas colunas (e.g., `vendas_2022`, `vendas_2023`).
*   **Long:** Formato Tidy, cada observação em uma linha.

#### `pd.melt()` (Wide para Long - Como `gather`)

In [118]:
df_wide = pd.DataFrame({
    'Aluno': ['João', 'Maria', 'Pedro'],
    'Nota_P1': [7.5, 8.0, 6.0],
    'Nota_P2': [8.5, 7.0, 9.0],
    'Nota_Trabalho': [9.0, 9.5, 8.0]
})
print("DataFrame Wide (Original):")
display(df_wide)

df_long = pd.melt(
    df_wide,
    id_vars=['Aluno'],                     # Coluna(s) identificadora(s)
    value_vars=['Nota_P1', 'Nota_P2', 'Nota_Trabalho'], # Colunas a empilhar
    var_name='Avaliacao',                # Nome da nova coluna com nomes antigos
    value_name='Nota'                    # Nome da nova coluna com valores
)

print("\nDataFrame Long (Após melt):")
# sort_values ajuda a visualizar a transformação
display(df_long.sort_values(by=['Aluno', 'Avaliacao']).reset_index(drop=True))

DataFrame Wide (Original):


,Aluno,Nota_P1,Nota_P2,Nota_Trabalho
0,João,7.5,8.5,9.0
1,Maria,8.0,7.0,9.5
2,Pedro,6.0,9.0,8.0



DataFrame Long (Após melt):


,Aluno,Avaliacao,Nota
0,João,Nota_P1,7.5
1,João,Nota_P2,8.5
2,João,Nota_Trabalho,9.0
3,Maria,Nota_P1,8.0
4,Maria,Nota_P2,7.0
5,Maria,Nota_Trabalho,9.5
6,Pedro,Nota_P1,6.0
7,Pedro,Nota_P2,9.0
8,Pedro,Nota_Trabalho,8.0


#### `df.pivot_table()` (Long para Wide - Como `spread`)

Usaremos `pivot_table` que é mais flexível que `pivot` pois permite agregação se houver linhas duplicadas para a mesma combinação de índice/coluna.

In [119]:
# Usando o df_long do exemplo anterior
print("DataFrame Long (Original):")
display(df_long.head())

try:
    df_pivoted = pd.pivot_table(
        df_long,
        index='Aluno',      # Coluna(s) que formarão o novo índice
        columns='Avaliacao', # Coluna cujos valores virarão novas colunas
        values='Nota',      # Coluna com os valores a preencher
        aggfunc='first'     # Função de agregação (se houver duplicatas, 'first' pega o primeiro valor)
                            # Se não houver duplicatas, a agregação não tem efeito prático além de permitir o uso.
    )
    # pivot_table coloca a coluna 'index' como índice, reset_index() a transforma de volta em coluna
    df_pivoted = df_pivoted.reset_index()
    # A ordem das colunas pode mudar, vamos reordenar para comparar com o original
    df_pivoted = df_pivoted[['Aluno', 'Nota_P1', 'Nota_P2', 'Nota_Trabalho']]
    # Remover o nome do índice das colunas (gerado pelo pivot_table)
    df_pivoted.columns.name = None

    print("\nDataFrame Wide (Após pivot_table):")
    display(df_pivoted)

except Exception as e:
    print(f"Erro ao executar pivot_table: {e}")

DataFrame Long (Original):


,Aluno,Avaliacao,Nota
0,João,Nota_P1,7.5
1,Maria,Nota_P1,8.0
2,Pedro,Nota_P1,6.0
3,João,Nota_P2,8.5
4,Maria,Nota_P2,7.0



DataFrame Wide (Após pivot_table):


,Aluno,Nota_P1,Nota_P2,Nota_Trabalho
0,João,7.5,8.5,9.0
1,Maria,8.0,7.0,9.5
2,Pedro,6.0,9.0,8.0


### 5.5. Separar e Unir Colunas

#### `str.split()` (Separar - Como `separate`)

In [120]:
df_sep = pd.DataFrame({'Codigo_Completo': ['PROD-A-10', 'PROD-B-25', 'SERV-C-05', 'PROD-D']}) # Adicionado um caso com menos partes
print("DataFrame Original:")
display(df_sep)

# Separa a coluna 'Codigo_Completo' em três novas colunas usando '-' como delimitador
# expand=True cria novas colunas no DataFrame
# O número de colunas criadas é baseado no máximo de splits encontrados
# Se uma linha tem menos splits, as colunas extras ficam com None
split_cols = df_sep['Codigo_Completo'].str.split('-', expand=True)

# Renomear as colunas geradas automaticamente (0, 1, 2...)
split_cols.columns = [f'parte_{i+1}' for i in range(split_cols.shape[1])]

# Juntar as novas colunas ao DataFrame original (ou substituir)
df_sep = pd.concat([df_sep, split_cols], axis=1)

print("\nDataFrame após split:")
display(df_sep)

DataFrame Original:


,Codigo_Completo
0,PROD-A-10
1,PROD-B-25
2,SERV-C-05
3,PROD-D



DataFrame após split:


,Codigo_Completo,parte_1,parte_2,parte_3
0,PROD-A-10,PROD,A,10
1,PROD-B-25,PROD,B,25
2,SERV-C-05,SERV,C,05
3,PROD-D,PROD,D,None


#### Unir Colunas (Como `unite`)

Geralmente feito com concatenação de strings.

In [121]:
df_unir = pd.DataFrame({
    'Prefixo': ['USR', 'ADM', 'USR'],
    'ID_Num': [101, 5, 22],
    'Status': ['Ativo', 'Inativo', 'Ativo']
})
print("DataFrame Original:")
display(df_unir)

# Unir colunas para formar um identificador único
# Importante converter números para string antes de concatenar
# str.zfill(3) garante que o ID tenha 3 dígitos com zeros à esquerda
df_unir['ID_Completo'] = df_unir['Prefixo'] + '_' + \
                        df_unir['ID_Num'].astype(str).str.zfill(3) + '_' + \
                        df_unir['Status']

print("\nDataFrame após unir colunas:")
display(df_unir)

DataFrame Original:


,Prefixo,ID_Num,Status
0,USR,101,Ativo
1,ADM,5,Inativo
2,USR,22,Ativo



DataFrame após unir colunas:


,Prefixo,ID_Num,Status,ID_Completo
0,USR,101,Ativo,USR_101_Ativo
1,ADM,5,Inativo,ADM_005_Inativo
2,USR,22,Ativo,USR_022_Ativo


## 6. Integração no Fluxo de Trabalho

Estas etapas (Coleta -> Importação -> Organização/Transformação) são interconectadas e muitas vezes iterativas. O objetivo é obter um DataFrame *Tidy* pronto para as próximas fases.

## 7. Conclusão

Nesta aula prática, cobrimos:
*   **Coleta:** Fontes de dados (conceito).
*   **Web Scraping:** Extração de HTML com `requests` e `BeautifulSoup` (e suas implicações éticas/legais).
*   **Importação:** Leitura de arquivos (`CSV`, `JSON`, `Excel`, `FWF`) e BD (`SQLite`) com `Pandas`, usando parâmetros para controle fino (`dtype`, `na_values`, `parse_dates`, etc.).
*   **Organização:** Inspeção inicial, renomeação, ajuste de tipos de dados.
*   **Transformação Estrutural:** Remodelagem (`melt`, `pivot_table`) e manipulação de colunas (`split`, concatenação) para alcançar o formato Tidy.

Dominar estas etapas é fundamental para garantir a qualidade e a adequação dos dados para análises subsequentes.

**Próximos Passos:** Limpeza de Dados (tratamento de nulos, outliers) e Análise Exploratória de Dados (EDA).